In [27]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [17]:
import cv2
from PIL import Image
import numpy as np
def np_array_to_cv2_array(np_array):
    return Image.fromarray(np_array.astype(np.uint8))
class HOG_Complexity:
    def __init__(self):
        self.hog_desc = cv2.HOGDescriptor()
        self.labelsScores = {}

    def calculate_score(self, trainloader):
        for images, labels in trainloader:
            for image in images:
                print(image.numpy().shape)
                n = self.hog_desc.compute(np_array_to_cv2_array(image.numpy()))
                print(n)


In [1]:
def vog_calc(x_train, y_train, model):
    vog = {}
    grad_X = x_train
    grad_Y = y_train
    grad_X.requires_grad = True
    sel_nodes_shape = grad_Y.shape
    ones = torch.ones(sel_nodes_shape)
    logits = model(grad_X)
    probs = torch.nn.Softmax(dim=1)(logits)
    sel_nodes = probs[torch.arange(len(grad_Y)), grad_Y.type(torch.LongTensor)]
    sel_nodes.backward(ones)
    grad = grad_X.grad.data.numpy()
    averageVOG = torch.zeros_like(grad_X[0])
    for i in range(grad_X.shape[0]):
        averageVOG += grad[i]
    averageVOG /= grad_X.shape[0]
    print(averageVOG)

In [2]:

import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

Using cache found in /home/niranjan/.cache/torch/hub/pytorch_vision_v0.10.0


In [3]:
from torchvision import transforms, datasets
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
     
trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=len(trainset),
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=len(testset),
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
model.eval()

x_train, y_train = next(iter(trainloader))
vog = vog_calc(x_train, y_train, model)
print(vog)

NameError: name 'model' is not defined